In [87]:
import pymongo
import pandas as pd
from pandas import DataFrame, Series
from pymongo import MongoClient
import json
import plotly as py
py.tools.set_credentials_file(username = 'jcalcutt', api_key = '0PZ56yzpReSTHOb3Ul6k')
import cufflinks as cf
import datetime
from bson.objectid import ObjectId
import numpy as np
import plotly.plotly as py

In [2]:
df1 = pd.read_pickle("C:\Users\Student\Documents\UKelections2017\positivetweetsuptodate")

In [4]:
df1[:10]

,Date,No. of positive tweets by date,Category,No. of positive tweets per party,Proportion of positive tweets per party
0,2017-05-11,34822,Conservative,3848,0.110505
1,2017-05-11,34822,Cymru,68,0.001953
2,2017-05-11,34822,Green,581,0.016685
3,2017-05-11,34822,Labour,26653,0.765407
4,2017-05-11,34822,LibDem,478,0.013727
5,2017-05-11,34822,SNP,2282,0.065533
6,2017-05-11,34822,UKIP,775,0.022256
7,2017-05-11,34822,other,137,0.003934
8,2017-05-12,109198,Conservative,15097,0.138253
9,2017-05-12,109198,Cymru,137,0.001255


In [5]:
df1 = df1.rename( columns={"No. of positive tweets per party": "count"})
df1 = df1.rename( columns={"Proportion of positive tweets per party": "proportion"})
df1['Date'] = pd.to_datetime(df1['Date'])
df1 = df1.rename( columns={"No. of positive tweets by date": "overall_count"})
df1 = df1.rename( columns={"Category": "party"})

In [7]:
df1[:10]

,Date,overall_count,party,count,proportion
0,2017-05-11,34822,Conservative,3848,0.110505
1,2017-05-11,34822,Cymru,68,0.001953
2,2017-05-11,34822,Green,581,0.016685
3,2017-05-11,34822,Labour,26653,0.765407
4,2017-05-11,34822,LibDem,478,0.013727
5,2017-05-11,34822,SNP,2282,0.065533
6,2017-05-11,34822,UKIP,775,0.022256
7,2017-05-11,34822,other,137,0.003934
8,2017-05-12,109198,Conservative,15097,0.138253
9,2017-05-12,109198,Cymru,137,0.001255


In [8]:
df2 = df1.pivot(index='Date', columns='party', values='count')

In [9]:
df2

party,Conservative,Cymru,Green,Labour,LibDem,SNP,UKIP,other
Date,,,,,,,,
2017-05-11,3848,68,581,26653,478,2282,775,137
2017-05-12,15097,137,1346,77089,1649,7002,6544,334
2017-05-13,23009,177,1571,69344,1663,9074,5854,439
2017-05-14,22149,114,881,72080,2925,11676,3640,620
2017-05-15,31824,314,837,67441,2055,7919,2158,290
2017-05-16,26375,754,705,158853,2829,9567,2773,631
2017-05-17,19980,499,919,101508,5354,9391,3564,997
2017-05-18,34147,607,3844,81387,4178,16066,7759,2285
2017-05-19,23749,209,1137,65528,1599,9418,5077,1857


In [11]:
dfcum = df2.cumsum()

In [12]:
dfcum

party,Conservative,Cymru,Green,Labour,LibDem,SNP,UKIP,other
Date,,,,,,,,
2017-05-11,3848,68,581,26653,478,2282,775,137
2017-05-12,18945,205,1927,103742,2127,9284,7319,471
2017-05-13,41954,382,3498,173086,3790,18358,13173,910
2017-05-14,64103,496,4379,245166,6715,30034,16813,1530
2017-05-15,95927,810,5216,312607,8770,37953,18971,1820
2017-05-16,122302,1564,5921,471460,11599,47520,21744,2451
2017-05-17,142282,2063,6840,572968,16953,56911,25308,3448
2017-05-18,176429,2670,10684,654355,21131,72977,33067,5733
2017-05-19,200178,2879,11821,719883,22730,82395,38144,7590


In [15]:
partycol = pd.DataFrame.from_csv('c:\Election\partycolours.txt', sep='#')

### Cumulative count of +ve tweets:

In [18]:
dfcum.iplot(y=['Conservative', 'Labour', 'LibDem','Green', 'UKIP'],colors = list(partycol['rgb'].values[:5]))

In [33]:
dfpct = df2.pct_change()
dfpct = dfpct[2:]

In [75]:
news = pd.read_csv('c:\\Election\\twitter-scrape-master\\newsheadlines.txt', sep='|')
news = news[:50]

In [85]:
news['timestamp'] = pd.to_datetime(news['timestamp'])
news2 = news.set_index("timestamp", drop=False)
news2
#news3 = news.reindex(dfpct.index, method='nearest')
#news3['e'] = pd.Series(3.5, index=df1.index)
news4 = DataFrame()
news4["headlines"] = '1) '+news2["head1"].map(str) + '<BR> '+  '2) '+news2["head2"] + '<BR>' + '3) '+news2['head3']
news4["date"] = news2["timestamp"]
news4['BBC headlines'] = pd.Series(3.5, index=news4.index)
news4[:3]

,headlines,date,BBC headlines
timestamp,,,
2017-05-18 09:21:45.530,1) Tory manifesto pledge on social care<BR> 2)...,2017-05-18 09:21:45.530,3.5
2017-05-18 09:21:50.145,1) Tory manifesto pledge on social care<BR> 2)...,2017-05-18 09:21:50.145,3.5
2017-05-18 09:21:55.163,1) Tory manifesto pledge on social care<BR> 2)...,2017-05-18 09:21:55.163,3.5


### Percentage change of +ve tweets

In [95]:
fig1 = dfpct.iplot(y=['Conservative', 'Labour', 'LibDem','Green', 'UKIP'],colors = list(partycol['rgb'].values[:5]), asFigure=True)
news4.iplot(x = ['date'], y = ['BBC headlines'],mode = 'markers',  text = 'headlines', asFigure = True )
fig1['data'].extend(fig2['data'])
py.iplot(fig1)

### Absolute +ve tweets for each party over time

In [92]:
df2.iplot()

In [42]:
dfdiff = df2.diff()
dfdiff[:10]

party,Conservative,Cymru,Green,Labour,LibDem,SNP,UKIP,other
Date,,,,,,,,
2017-05-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05-12,11249.0,69.0,765.0,50436.0,1171.0,4720.0,5769.0,197.0
2017-05-13,7912.0,40.0,225.0,-7745.0,14.0,2072.0,-690.0,105.0
2017-05-14,-860.0,-63.0,-690.0,2736.0,1262.0,2602.0,-2214.0,181.0
2017-05-15,9675.0,200.0,-44.0,-4639.0,-870.0,-3757.0,-1482.0,-330.0
2017-05-16,-5449.0,440.0,-132.0,91412.0,774.0,1648.0,615.0,341.0
2017-05-17,-6395.0,-255.0,214.0,-57345.0,2525.0,-176.0,791.0,366.0
2017-05-18,14167.0,108.0,2925.0,-20121.0,-1176.0,6675.0,4195.0,1288.0
2017-05-19,-10398.0,-398.0,-2707.0,-15859.0,-2579.0,-6648.0,-2682.0,-428.0


### Difference in +ve tweets from following value:

In [67]:
dfdiff.iplot(kind='bar', y=['Labour','Conservative'], yTitle='difference in +ve tweets' )

In [48]:
mean = df2.mean()

In [49]:
mean

party
Conservative    21302.230769
Cymru             245.307692
Green            1190.153846
Labour          84583.538462
LibDem           2032.000000
SNP              8840.307692
UKIP             3918.153846
other            1127.846154
dtype: float64

In [51]:
mean.Conservative

21302.23076923077

In [56]:
df2['conav'] = pd.Series(mean.Conservative, index=df2.index)
df2['Labav'] = pd.Series(mean.Labour, index=df2.index)
df2['LDav'] = pd.Series(mean.LibDem, index=df2.index)

In [57]:
df2

party,Conservative,Cymru,Green,Labour,LibDem,SNP,UKIP,other,conav,Labav,LDav
Date,,,,,,,,,,,
2017-05-11,3848,68,581,26653,478,2282,775,137,21302.230769,84583.538462,2032.0
2017-05-12,15097,137,1346,77089,1649,7002,6544,334,21302.230769,84583.538462,2032.0
2017-05-13,23009,177,1571,69344,1663,9074,5854,439,21302.230769,84583.538462,2032.0
2017-05-14,22149,114,881,72080,2925,11676,3640,620,21302.230769,84583.538462,2032.0
2017-05-15,31824,314,837,67441,2055,7919,2158,290,21302.230769,84583.538462,2032.0
2017-05-16,26375,754,705,158853,2829,9567,2773,631,21302.230769,84583.538462,2032.0
2017-05-17,19980,499,919,101508,5354,9391,3564,997,21302.230769,84583.538462,2032.0
2017-05-18,34147,607,3844,81387,4178,16066,7759,2285,21302.230769,84583.538462,2032.0
2017-05-19,23749,209,1137,65528,1599,9418,5077,1857,21302.230769,84583.538462,2032.0


In [59]:
df3=DataFrame()

In [64]:
df3['Con'] = df2['Conservative'] - df2['conav']
df3['Lab'] = df2['Labour'] - df2['Labav']
df3['LibDem'] = df2['LibDem'] - df2['LDav']

In [65]:
df3

,Con,Lab,LibDem
Date,,,
2017-05-11,-17454.230769,-57930.538462,-1554.0
2017-05-12,-6205.230769,-7494.538462,-383.0
2017-05-13,1706.769231,-15239.538462,-369.0
2017-05-14,846.769231,-12503.538462,893.0
2017-05-15,10521.769231,-17142.538462,23.0
2017-05-16,5072.769231,74269.461538,797.0
2017-05-17,-1322.230769,16924.461538,3322.0
2017-05-18,12844.769231,-3196.538462,2146.0
2017-05-19,2446.769231,-19055.538462,-433.0


### Difference in +ve tweets from mean +ve tweets over entire time range:

In [69]:
df3.iplot(kind = 'bar',y=['Con', 'Lab', 'LibDem'],colors = list(partycol['rgb'].values[:3]))